In [ ]:
# Load preprocessed datasets



import pandas as pd

data_path = "../data/"
X_train = pd.read_csv(data_path + "X_train.csv")
X_val = pd.read_csv(data_path + "X_val.csv")
y_train = pd.read_csv(data_path + "y_train.csv").squeeze()  # Convert to Series
y_val = pd.read_csv(data_path + "y_val.csv").squeeze()  # Convert to Series
X_test = pd.read_csv(data_path + "X_test.csv")
test_merged.to_csv("../data/test_merged.csv", index=False)



In [ ]:
# Recreate test_merged
test_merged = datasets['test'].merge(
    datasets['meal_info'], on='meal_id', how='left'
).merge(
    datasets['fulfilment_center_info'], on='center_id', how='left'
)

# Add derived features
test_merged['price_diff'] = test_merged['base_price'] - test_merged['checkout_price']
test_merged['promotion'] = test_merged['emailer_for_promotion'] + test_merged['homepage_featured']

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder
for col in ['category', 'cuisine', 'center_type']:
    le = LabelEncoder()
    test_merged[col] = le.fit_transform(test_merged[col])

In [ ]:
# Define feature columns for test data
feature_cols = [
    "checkout_price", "base_price", "price_diff", "promotion", 
    "category", "cuisine", "city_code", "region_code", "center_type", "op_area"
]

# Prepare X_test
X_test = test_merged[feature_cols]

# Preview X_test
print("X_test - Overview:")
print(X_test.head())


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [4]:
# Load preprocessed datasets
X_train = pd.read_csv("../data/X_train.csv")
X_val = pd.read_csv("../data/X_val.csv")
y_train = pd.read_csv("../data/y_train.csv").squeeze()  # Convert to Series
y_val = pd.read_csv("../data/y_val.csv").squeeze()  # Convert to Series

# Verify shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


X_train shape: (365238, 10), y_train shape: (365238,)
X_val shape: (91310, 10), y_val shape: (91310,)


In [6]:
# Load or prepare test_merged
test_merged = pd.read_csv("../data/test_merged.csv")

# Define feature columns for test data
feature_cols = [
    "checkout_price", "base_price", "price_diff", "promotion", 
    "category", "cuisine", "city_code", "region_code", "center_type", "op_area"
]

# Prepare X_test
X_test = test_merged[feature_cols]

# Preview X_test
print("X_test - Overview:")
print(X_test.head())


X_test - Overview:
   checkout_price  base_price  price_diff  promotion  category  cuisine  \
0          158.11      159.11         1.0          0         0        3   
1          160.11      159.11        -1.0          0         0        3   
2          157.14      159.14         2.0          0         0        3   
3          162.02      162.02         0.0          0         0        1   
4          163.93      163.93         0.0          0         0        1   

   city_code  region_code  center_type  op_area  
0        647           56            2      2.0  
1        647           56            2      2.0  
2        647           56            2      2.0  
3        647           56            2      2.0  
4        647           56            2      2.0  


In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load preprocessed training and validation datasets
data_path = "../data/"
X_train = pd.read_csv(data_path + "X_train.csv")
X_val = pd.read_csv(data_path + "X_val.csv")
y_train = pd.read_csv(data_path + "y_train.csv").squeeze()  # Convert to Series
y_val = pd.read_csv(data_path + "y_val.csv").squeeze()  # Convert to Series

# Load and prepare test data
test_merged = pd.read_csv(data_path + "test_merged.csv")
feature_cols = [
    "checkout_price", "base_price", "price_diff", "promotion", 
    "category", "cuisine", "city_code", "region_code", "center_type", "op_area"
]
X_test = test_merged[feature_cols]

# Initialize and train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Predict on validation set
y_val_rf_pred = rf_model.predict(X_val)

# Evaluate the model on validation set
mae_rf = mean_absolute_error(y_val, y_val_rf_pred)
rmse_rf = np.sqrt(mean_squared_error(y_val, y_val_rf_pred))
r2_rf = r2_score(y_val, y_val_rf_pred)

print("\nRandom Forest Model Evaluation on Validation Set:")
print(f"Mean Absolute Error (MAE): {mae_rf}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf}")
print(f"R-squared (R2): {r2_rf}")

# Predict on test set
y_test_rf_pred = rf_model.predict(X_test)

# Save test predictions
test_predictions = test_merged[["id"]].copy()
test_predictions["num_orders"] = y_test_rf_pred
test_predictions.to_csv("../results/test_predictions_rf.csv", index=False)

print("\nRandom Forest test predictions saved to ../results/test_predictions_rf.csv")



Random Forest Model Evaluation on Validation Set:
Mean Absolute Error (MAE): 107.10574685330496
Root Mean Squared Error (RMSE): 210.81287030422902
R-squared (R2): 0.7086405655821549

Random Forest test predictions saved to ../results/test_predictions_rf.csv
